# 2024/06/05 LODES User Manual
_Author: Meaghan Freund_

### **Introduction**
LODES (LEHD Origin-Destination Employment Statistics) is a collection of state-organized data files, owned by the U.S. Census Bureau and is a part of the LEHD (Longitudinal Employer-Household Data) program. The datasets are collected through LEHD Census data, which is made through combining previously collected survey and administrative data from participating states on jobs, businesses, and workers. 

### **Data Collection and Updates**

LODES data is collected and mapped at the smallest geographic unit that the Census Bureau collects: blocks. This data is collected annually and updated regularly through new versions of LODES. The most recent version, LODES8, spans from 2002-2021 inclusively and maps all blocks with 2020 Census delineations, including blocks that were originally defined in past Census delineations, such as from the 2010 Census.

### **Geographic Coverage**

LODES covers 51 states in the U.S., including defining Washington D.C. as its own state, and territorial partners such as Puerto Rico and the Virgin Islands. However, in LODES8, Puerto Rico is the only territorial partner that is covered. Similarly, certain states do not have files in specific years that LODES provides. Some of the reasons can include a lack of availability of data for a state or a location not being in regular production for LODES.
The current list of exceptions includes:
- *2002*: Arkansas, Arizona, DC, Massachusetts, Mississippi, New Hampshire
- *2003*: Arizona, DC, Massachusetts, Mississippi
- *2004-2009*: DC, Massachusetts
- *2010*: Massachusetts
- *2017-2018*: Alaska
- *2019-2021*: Alaska, Arkansas, Mississippi

These intersections do lack information concerning other files within LODES, but for the ADRIO maker, these exceptions are omitted from the calls altogether.

### **Data Files**

Of the three files available in LODES (RAC, WAC, and OD) the ADRIO maker relies on the Origin-Destination files to map worker flows between residence and workplace locations through matrices. The Origin-Destination files provide the number of individuals who live in a specific home GEOID and commute to a corresponding work GEOID.

### **Additional Resources**

For more detailed information:
- Visit the [LEHD Data Page](https://lehd.ces.census.gov/data/#lodes) on the Census Bureau's website concerning the LODES files.
- Refer to the documentation for the most recent version, [LODES8.1](https://lehd.ces.census.gov/data/lodes/LODES8/LODESTechDoc8.1.pdf).

### **Basic Queries**

The Commuters and Geoid calls are two simple yet imperative queries that show the basic functionality of the LODES ADRIO maker. The Geoid query represents the GEOIDs that are involved with the commuter matrices. The input given by the user for the scope, in this case being states, is translated into a list of GEOIDs. The Commuters query shows the total number of workers moving from a home GEOID to a work GEOID as a matrix. The matrix is read so that the rows represent the residence GEOID and the columns are the work location GEOID.

In [1]:
import numpy as np

from epymorph.geography.us_census import StateScope
from epymorph import *
from epymorph.adrio import acs5
from epymorph.adrio import lodes
from epymorph.data.ipm.no import No as NoIpm
from epymorph.data.mm.no import No as NoMm


state_scope = StateScope.in_states(["AZ", "CO", "NV", "NM"], 2020)
time_period = 2015
geoids = state_scope.node_ids

commuters_adrio = lodes.Commuters(time_period)

rume = SingleStrataRume.build(
    ipm=NoIpm(),
    mm=NoMm(),
    init=init.NoInfection(),
    scope=state_scope,
    time_frame=TimeFrame.year(time_period),
    params={
        "population": acs5.Population(),
        "commuters": commuters_adrio,
    },
)

rume.estimate_data()

ADRIO data usage estimation:
- epymorph.adrio.acs5.Population (no estimate available)
- epymorph.adrio.lodes.Commuters will be pulled from cache
In total we will:
- Download no additional data
- Write no new data to disk cache


In [2]:
with sim_messaging():
    commuters = commuters_adrio.with_context(scope=state_scope).evaluate()

Loading epymorph.adrio.lodes.Commuters:
  |####################| 100%  (7.113s)


In [3]:
print(f"Home/Work GEOIDs:\n {geoids}\n")

print(f"Commuters Matrix:\n {commuters}\n")

Home/Work GEOIDs:
 ['04' '08' '32' '35']

Commuters Matrix:
 [[2550132    2582   13263    8100]
 [   1202 2405258     382    5557]
 [   3552     535 1179411     361]
 [   6813    4824     409  764244]]



# Attributes

The Commuters class outputs the total number of workers commuting, but LODES provides three categories for attributes specifying the type of workers: Age, Monthly Income, and Industry Sectors. Within each category, there are three ranges within them, and the sum of the ranges equals the total number of workers. All of these categories and the total commuters are displayed as NxN matrices of integers, excluding the Geoid query.

## Age
- '29 and Under'
    -  Commuters that are ages 29 and under.
- ''30_54'
    -  Commuters that are between the ages of 30 and 54.
- '55 and Over'
    -  Commuters that are ages 55 and over.

## Monthly Income
- '$1250 and Under'
    -  Commuters that earn $1250 and under per month.
- '$1251_$3333'
    -  Commuters that earn between $1251 and $3333 per month.
- '$3333 and Over'
    -  Commuters that earn over $3333 per month.

## Industry Sector
- 'Goods Producing'
    -  Commuters that work in Goods Producing industry sectors.
- 'Trade Transport Utility'
    -  Commuters that work in Trade, Transportation, and Utility industry sectors.
- 'Other'
    -  Commuters that work under all other service industry sectors other than the above claimed industries.


## Job Type
Along with the above categories, LODES provides files detailing different job types and the total number of jobs under that type. However, unlike the attributes, these matrices do not sum to be the total number of workers.

- 'All Jobs'
    -  All jobs regardless of job type. Allows for multiple jobs per person and is the default when calling the above attributes.
- 'Primary Jobs'
    -  Primary jobs, which a primary job is the highest paying job for an individual worker for the year. Limits to one job per worker.
- 'All Private Jobs'
    -  All private jobs, which are privately owned businesses and organizations excluding federal government jobs.
- 'Private Primary Jobs'
    -  Primary jobs within the private sector.
- 'All Federal Jobs'
    -  All jobs within the federal government sector.
- 'Federal Primary Jobs
    -  Jobs under the federal government sector that are defined as primary jobs.

Job type is an additional variable that can be combined with the previously explained attributes. For example, a user may retrieve a matrix of commuters work for the Goods Producing industry sector for only Primary jobs. This variable is not required for all calls and the default value will be 'All Jobs'.

### Age Range Attribute Example

Below is an example of calling the three different age ranges provided by LODES in a geo spec. The example here loads four counties into the matrices rather than the four states that was used previously.

In [4]:
from epymorph.geography.us_census import CountyScope
from epymorph import *
from epymorph.adrio import acs5
from epymorph.adrio import lodes

time_period = 2015
county_scope = CountyScope.in_counties(["04013", "08041", "32003", "35001"], year=2020)
geoids = county_scope.node_ids

commuters_29_under_adrio = lodes.CommutersByAge(time_period, "29 and Under")
commuters_30_54_adrio = lodes.CommutersByAge(time_period, "30_54")
commuters_55_over_adrio = lodes.CommutersByAge(time_period, "55 and Over")

rume = SingleStrataRume.build(
    ipm=NoIpm(),
    mm=NoMm(),
    init=init.NoInfection(),
    scope=county_scope,
    time_frame=TimeFrame.year(time_period),
    params={
        "population": acs5.Population(),
        "commuters_29_under": commuters_29_under_adrio,
        "commuters_30_54": commuters_30_54_adrio,
        "commuters_55_over": commuters_55_over_adrio,
    },
)

rume.estimate_data()

ADRIO data usage estimation:
- epymorph.adrio.acs5.Population (no estimate available)
- epymorph.adrio.lodes.CommutersByAge will be pulled from cache
- epymorph.adrio.lodes.CommutersByAge will be pulled from cache
- epymorph.adrio.lodes.CommutersByAge will be pulled from cache
In total we will:
- Download no additional data
- Write no new data to disk cache


In [5]:
with sim_messaging():
    commuters_29_under = commuters_29_under_adrio.with_context(
        scope=county_scope
    ).evaluate()
    commuters_30_54 = commuters_30_54_adrio.with_context(scope=county_scope).evaluate()
    commuters_55_over = commuters_55_over_adrio.with_context(
        scope=county_scope
    ).evaluate()

Loading epymorph.adrio.lodes.CommutersByAge:
  |####################| 100%  (5.839s)
Loading epymorph.adrio.lodes.CommutersByAge:
  |####################| 100%  (5.998s)
Loading epymorph.adrio.lodes.CommutersByAge:
  |####################| 100%  (5.848s)


In [6]:
print(f"Commuters ages 29 and under:\n {commuters_29_under}\n")
print(f"Commuters between ages 30 and 54:\n {commuters_30_54}\n")
print(f"Commuters ages 55 and over:\n {commuters_55_over}\n")

Commuters ages 29 and under:
 [[369848     11    287    195]
 [    12  43020      9     29]
 [   245     11 186798     48]
 [   119     26     41  55536]]

Commuters between ages 30 and 54:
 [[369848     11    287    195]
 [    12  43020      9     29]
 [   245     11 186798     48]
 [   119     26     41  55536]]

Commuters ages 55 and over:
 [[369848     11    287    195]
 [    12  43020      9     29]
 [   245     11 186798     48]
 [   119     26     41  55536]]

